In [1]:
import pandas as pd 
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
#ccba
ccba = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_ccba_full_hashed.csv')
#cdtx
cdtx = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_cdtx0001_full_hashed.csv')
#custinfo
cdtx2 = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_custinfo_full_hashed.csv')
#cdtx
dp = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_dp_full_hashed.csv')
#remit
remit = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_remit1_full_hashed.csv')
#alert_time
alert_time = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_x_alert_date.csv')
#y
y = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\train_y_answer.csv')
#案件名單
doc = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\預測的案件名單及提交檔案範例.csv')

In [3]:
remit['trade_amount_usd'] = remit['trade_amount_usd'] * 30

In [4]:
ccba

,cust_id,lupay,byymm,cycam,usgam,clamt,csamt,inamt,cucsm,cucah
0,0172056578071e83399216fcd640bdc4de4583149d0fd9...,0.0,0,152628.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0172056578071e83399216fcd640bdc4de4583149d0fd9...,0.0,30,152628.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0172056578071e83399216fcd640bdc4de4583149d0fd9...,0.0,61,152628.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0172056578071e83399216fcd640bdc4de4583149d0fd9...,0.0,91,152628.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0172056578071e83399216fcd640bdc4de4583149d0fd9...,0.0,122,152628.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
59070,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,0.0,244,781570.0,63952.0,0.0,0.0,0.0,61927.0,0.0
59071,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,61927.0,275,781570.0,37297.0,0.0,0.0,0.0,37276.0,0.0
59072,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,37297.0,306,781570.0,45413.0,0.0,0.0,0.0,19766.0,0.0
59073,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,19766.0,334,781570.0,26360.0,0.0,0.0,1142.0,2568.0,0.0


In [5]:
doc_merged = cdtx2.merge(doc, on='alert_key', how='right').merge(alert_time, on='alert_key', how='left')
doc_merged = doc_merged[~doc_merged['cust_id'].isnull()]
max_doc = doc_merged.groupby('cust_id')['date'].max().reset_index()
max_doc.rename(columns={'date':'max_date'},inplace=True)
doc_merged = doc_merged.merge(max_doc, on='cust_id', how='left')
prev_list = doc_merged[doc_merged['max_date']>doc_merged['date']]['alert_key'].tolist()

In [6]:
alert_remit = remit[remit['cust_id'].isin(cdtx2[cdtx2['alert_key'].isin(alert_time['alert_key'].tolist())]['cust_id'].tolist())]
train_remit = remit[~remit['cust_id'].isin(cdtx2[cdtx2['alert_key'].isin(alert_time['alert_key'].tolist())]['cust_id'].tolist())]
train_remit['y'] = 0

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
sar_idx = train_remit[train_remit['cust_id'].isin(cdtx2[cdtx2['alert_key'].isin(y[y['sar_flag']==1]['alert_key'].tolist())]['cust_id'].tolist())].index
train_remit.loc[sar_idx, 'y'] = 1

C:\Users\rocker\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [9]:
train_remit[train_remit.cust_id == 'deea7839000d9f8c1e3fed53e0a1141266fb8d7149a05008bc4de475b8785c64']

,cust_id,trans_date,trans_no,trade_amount_usd,y


In [64]:
alert_time = cdtx2.merge(alert_time)[['alert_key', 'cust_id','date']]
alert_time.columns = ['alert_key', 'cust_id', 'tx_date']
alert_remit = alert_remit.merge(alert_time)

In [65]:
(train_remit.y == 1).sum()

4995

In [67]:
#Data Cleansing
#SAR戶僅留前X天
def sar_pred_d(x, day):
  prev_d = x.groupby('cust_id')['trans_date'].max() - day
  prev_d = prev_d.reset_index()
  prev_d.rename(columns={'trans_date':'prev_d'}, inplace=True)
  x = x.merge(prev_d, on='cust_id', how='left')
  x.drop(x[x['trans_date']<x['prev_d']].index,inplace=True)
  x.pop('prev_d')
  return x

train_remit = sar_pred_d(train_remit, 30)
alert_remit = sar_pred_d(alert_remit, 30)


In [68]:
train_remit = pd.merge(train_remit, cdtx2, how ='left')
alert_remit = pd.merge(alert_remit, cdtx2, how ='left')

In [69]:
train_remit

,cust_id,trans_date,trans_no,trade_amount_usd,y,alert_key,risk_rank,occupation_code,total_asset,AGE
0,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,4,10309410.0,0,240088,3,15.0,2126903.0,4
1,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,4,10309410.0,0,292345,3,15.0,980597.0,4
2,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,4,10309410.0,0,294181,3,15.0,980597.0,4
3,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,4,10309410.0,0,294760,3,15.0,980597.0,4
4,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,356,0,63270.0,0,218725,3,5.0,1817475.0,2
...,...,...,...,...,...,...,...,...,...,...
18334,ff6b0040005b54f4b7955942e74a0a9ec466abb2a017a5...,312,0,996270.0,0,286951,1,5.0,2293.0,2
18335,ff6b0040005b54f4b7955942e74a0a9ec466abb2a017a5...,312,0,996270.0,0,292611,1,5.0,2293.0,2
18336,ff6c1217b5e039fda46bf4dfba2d54291154a1db54c677...,386,0,2647080.0,0,316858,1,12.0,106840.0,3
18337,ff6c1217b5e039fda46bf4dfba2d54291154a1db54c677...,386,0,2647080.0,0,323174,1,12.0,1107479.0,3


In [70]:
(train_remit.y == 1).sum()

2409

In [71]:
class renamer():
    def __init__(self):
        self.d = dict()
    def __call__(self, x):
        if x not in self.d:
            self.d[x] =  0
            return x
        else:
            self.d[x] += 1
            return "%s_%d" % (x, self.d[x])

In [72]:
def total_amt_rolling_3(data):
    total_amt_rolling_3 = (data[data.trans_no == 0].groupby(['cust_id', 'alert_key'])
                   .apply(lambda g: g.rolling(3, on='trans_date')['trade_amount_usd'].sum())).reset_index().drop(['level_2'], axis=1)
    total_amt_rolling_3.columns = ['cust_id', 'alert_key', 'total_amt_rolling_3']
    on_day_total_amt = data.groupby(['cust_id', 'trans_date', 'alert_key'])['trade_amount_usd'].sum().reset_index()
    on_day_total_amt.columns = ['cust_id', 'trans_date', 'alert_key', 'on_day_total_amt']
    data = pd.concat([data, total_amt_rolling_3], axis=1)
    data = data.rename(columns=renamer())  
    data = pd.merge(data, on_day_total_amt, how='left') 
    data['total_amt_rolling_3'] = data.apply(lambda x: x['on_day_total_amt'] if np.isnan(x['total_amt_rolling_3']) else x['total_amt_rolling_3'], axis=1)
    return data

train_remit = total_amt_rolling_3(train_remit)
alert_remit = total_amt_rolling_3(alert_remit)

In [73]:
def total_amt_rolling_7(data):
    total_amt_rolling_7 = (data[data.trans_no == 0].groupby(['cust_id', 'alert_key'])
                   .apply(lambda g: g.rolling(7, on='trans_date')['trade_amount_usd'].sum())).reset_index().drop(['level_2'], axis=1)
    total_amt_rolling_7.columns = ['cust_id', 'alert_key', 'total_amt_rolling_7']
    data = pd.concat([data, total_amt_rolling_7], axis=1)
    data = data.rename(columns=renamer())
    data['total_amt_rolling_7'] = data.apply(lambda x: x['on_day_total_amt'] if np.isnan(x['total_amt_rolling_7']) else x['total_amt_rolling_7'], axis=1)
    return data
train_remit = total_amt_rolling_7(train_remit)
alert_remit = total_amt_rolling_7(alert_remit)

In [74]:
def total_amt_rolling_14(data):
    total_amt_rolling_14 = (data[data.trans_no == 0].groupby(['cust_id', 'alert_key'])
                   .apply(lambda g: g.rolling(14, on='trans_date')['trade_amount_usd'].sum())).reset_index().drop(['level_2'], axis=1)
    total_amt_rolling_14.columns = ['cust_id', 'alert_key', 'total_amt_rolling_14']
    data = pd.concat([data, total_amt_rolling_14], axis=1)
    data = data.rename(columns=renamer())
    data['total_amt_rolling_14'] = data.apply(lambda x: x['on_day_total_amt'] if np.isnan(x['total_amt_rolling_14']) else x['total_amt_rolling_14'], axis=1)
    return data
train_remit = total_amt_rolling_14(train_remit)
alert_remit = total_amt_rolling_14(alert_remit)

In [75]:
train_remit

,cust_id,trans_date,trans_no,trade_amount_usd,y,alert_key,risk_rank,occupation_code,total_asset,AGE,cust_id_1,alert_key_1,total_amt_rolling_3,on_day_total_amt,cust_id_1_1,alert_key_1_1,total_amt_rolling_7,cust_id_1,alert_key_1,total_amt_rolling_14
0,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,4,10309410.0,0,240088,3,15.0,2126903.0,4,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,218725.0,10309410.0,10309410.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,218725.0,10309410.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,218725.0,10309410.0
1,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,4,10309410.0,0,292345,3,15.0,980597.0,4,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,218725.0,10309410.0,10309410.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,218725.0,10309410.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,218725.0,10309410.0
2,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,4,10309410.0,0,294181,3,15.0,980597.0,4,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219204.0,10309410.0,10309410.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219204.0,10309410.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219204.0,10309410.0
3,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,4,10309410.0,0,294760,3,15.0,980597.0,4,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219204.0,10309410.0,10309410.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219204.0,10309410.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219204.0,10309410.0
4,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,356,0,63270.0,0,218725,3,5.0,1817475.0,2,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219678.0,63270.0,63270.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219678.0,63270.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219678.0,63270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18334,ff6b0040005b54f4b7955942e74a0a9ec466abb2a017a5...,312,0,996270.0,0,286951,1,5.0,2293.0,2,NaN,NaN,996270.0,996270.0,NaN,NaN,996270.0,NaN,NaN,996270.0
18335,ff6b0040005b54f4b7955942e74a0a9ec466abb2a017a5...,312,0,996270.0,0,292611,1,5.0,2293.0,2,NaN,NaN,996270.0,996270.0,NaN,NaN,996270.0,NaN,NaN,996270.0
18336,ff6c1217b5e039fda46bf4dfba2d54291154a1db54c677...,386,0,2647080.0,0,316858,1,12.0,106840.0,3,NaN,NaN,2647080.0,2647080.0,NaN,NaN,2647080.0,NaN,NaN,2647080.0
18337,ff6c1217b5e039fda46bf4dfba2d54291154a1db54c677...,386,0,2647080.0,0,323174,1,12.0,1107479.0,3,NaN,NaN,2647080.0,2647080.0,NaN,NaN,2647080.0,NaN,NaN,2647080.0


In [76]:
train_remit['risk_rank'] = train_remit['risk_rank'].astype('category')
train_remit['occupation_code'] = train_remit['occupation_code'].astype('category')
train_remit['AGE'] = train_remit['AGE'].astype('category')
alert_remit['risk_rank'] = alert_remit['risk_rank'].astype('category')
alert_remit['occupation_code'] = alert_remit['occupation_code'].astype('category')
alert_remit['AGE'] = alert_remit['AGE'].astype('category')

In [77]:
train_remit['ratio_total_aum_7'] = train_remit.total_amt_rolling_7 /train_remit.total_asset
train_remit['ratio_total_aum_1'] = train_remit.on_day_total_amt  /train_remit.total_asset 
train_remit['ratio_total_aum_14'] = train_remit.total_amt_rolling_14/train_remit.total_asset
train_remit['ratio_total_aum_3'] = train_remit.total_amt_rolling_3  /train_remit.total_asset
alert_remit['ratio_total_aum_7'] = alert_remit.total_amt_rolling_7 /alert_remit.total_asset
alert_remit['ratio_total_aum_1'] = alert_remit.on_day_total_amt  /alert_remit.total_asset 
alert_remit['ratio_total_aum_14'] = alert_remit.total_amt_rolling_14/alert_remit.total_asset
alert_remit['ratio_total_aum_3'] = alert_remit.total_amt_rolling_3  /alert_remit.total_asset

In [78]:
alert_remit.replace([np.inf, -np.inf], 1, inplace=True)

In [79]:
train_remit.columns

Index(['cust_id', 'trans_date', 'trans_no', 'trade_amount_usd', 'y',
       'alert_key', 'risk_rank', 'occupation_code', 'total_asset', 'AGE',
       'cust_id_1', 'alert_key_1', 'total_amt_rolling_3', 'on_day_total_amt',
       'cust_id_1_1', 'alert_key_1_1', 'total_amt_rolling_7', 'cust_id_1',
       'alert_key_1', 'total_amt_rolling_14', 'ratio_total_aum_7',
       'ratio_total_aum_1', 'ratio_total_aum_14', 'ratio_total_aum_3'],
      dtype='object')

In [80]:
(train_remit.y == 1).sum()

2409

In [81]:
train_remit

,cust_id,trans_date,trans_no,trade_amount_usd,y,alert_key,risk_rank,occupation_code,total_asset,AGE,...,cust_id_1_1,alert_key_1_1,total_amt_rolling_7,cust_id_1,alert_key_1,total_amt_rolling_14,ratio_total_aum_7,ratio_total_aum_1,ratio_total_aum_14,ratio_total_aum_3
0,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,4,10309410.0,0,240088,3,15.0,2126903.0,4,...,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,218725.0,10309410.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,218725.0,10309410.0,4.847146,4.847146,4.847146,4.847146
1,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,4,10309410.0,0,292345,3,15.0,980597.0,4,...,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,218725.0,10309410.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,218725.0,10309410.0,10.513402,10.513402,10.513402,10.513402
2,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,4,10309410.0,0,294181,3,15.0,980597.0,4,...,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219204.0,10309410.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219204.0,10309410.0,10.513402,10.513402,10.513402,10.513402
3,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,4,10309410.0,0,294760,3,15.0,980597.0,4,...,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219204.0,10309410.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219204.0,10309410.0,10.513402,10.513402,10.513402,10.513402
4,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,356,0,63270.0,0,218725,3,5.0,1817475.0,2,...,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219678.0,63270.0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,219678.0,63270.0,0.034812,0.034812,0.034812,0.034812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18334,ff6b0040005b54f4b7955942e74a0a9ec466abb2a017a5...,312,0,996270.0,0,286951,1,5.0,2293.0,2,...,NaN,NaN,996270.0,NaN,NaN,996270.0,434.483210,434.483210,434.483210,434.483210
18335,ff6b0040005b54f4b7955942e74a0a9ec466abb2a017a5...,312,0,996270.0,0,292611,1,5.0,2293.0,2,...,NaN,NaN,996270.0,NaN,NaN,996270.0,434.483210,434.483210,434.483210,434.483210
18336,ff6c1217b5e039fda46bf4dfba2d54291154a1db54c677...,386,0,2647080.0,0,316858,1,12.0,106840.0,3,...,NaN,NaN,2647080.0,NaN,NaN,2647080.0,24.776114,24.776114,24.776114,24.776114
18337,ff6c1217b5e039fda46bf4dfba2d54291154a1db54c677...,386,0,2647080.0,0,323174,1,12.0,1107479.0,3,...,NaN,NaN,2647080.0,NaN,NaN,2647080.0,2.390185,2.390185,2.390185,2.390185


In [82]:
f = pd.read_csv(r'remit_test_feature.csv')[['cust_id', 'trans_date', 'over2']]

In [83]:
train_remit = train_remit.merge(f, how='left', on=['cust_id', 'trans_date'])
alert_remit = alert_remit.merge(f, how='left', on=['cust_id', 'trans_date'])

In [84]:
train_col = ['cust_id', 'trans_date','alert_key', 'trans_no', 'trade_amount_usd'
             #, 'country', 'cur_type'
             #, 'RISK_RANK', 'OCCUPATION_CODE'
             #, 'TOTAL_ASSET', 'AGE'
             , 'total_amt_rolling_3', 'on_day_total_amt'
             , 'total_amt_rolling_7', 'total_amt_rolling_14', 'ratio_total_aum_7',
       'ratio_total_aum_1', 'ratio_total_aum_14', 'ratio_total_aum_3', 'over2'
       , 'y']
x_col = train_col[3:-1]
train_data = train_remit[train_col]
train_data

,cust_id,trans_date,alert_key,trans_no,trade_amount_usd,total_amt_rolling_3,on_day_total_amt,total_amt_rolling_7,total_amt_rolling_14,ratio_total_aum_7,ratio_total_aum_1,ratio_total_aum_14,ratio_total_aum_3,over2,y
0,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,240088,4,10309410.0,10309410.0,10309410.0,10309410.0,10309410.0,4.847146,4.847146,4.847146,4.847146,0.0,0
1,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,292345,4,10309410.0,10309410.0,10309410.0,10309410.0,10309410.0,10.513402,10.513402,10.513402,10.513402,0.0,0
2,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,294181,4,10309410.0,10309410.0,10309410.0,10309410.0,10309410.0,10.513402,10.513402,10.513402,10.513402,0.0,0
3,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,153,294760,4,10309410.0,10309410.0,10309410.0,10309410.0,10309410.0,10.513402,10.513402,10.513402,10.513402,0.0,0
4,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,356,218725,0,63270.0,63270.0,63270.0,63270.0,63270.0,0.034812,0.034812,0.034812,0.034812,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18396,ff6b0040005b54f4b7955942e74a0a9ec466abb2a017a5...,312,286951,0,996270.0,996270.0,996270.0,996270.0,996270.0,434.483210,434.483210,434.483210,434.483210,NaN,0
18397,ff6b0040005b54f4b7955942e74a0a9ec466abb2a017a5...,312,292611,0,996270.0,996270.0,996270.0,996270.0,996270.0,434.483210,434.483210,434.483210,434.483210,NaN,0
18398,ff6c1217b5e039fda46bf4dfba2d54291154a1db54c677...,386,316858,0,2647080.0,2647080.0,2647080.0,2647080.0,2647080.0,24.776114,24.776114,24.776114,24.776114,NaN,0
18399,ff6c1217b5e039fda46bf4dfba2d54291154a1db54c677...,386,323174,0,2647080.0,2647080.0,2647080.0,2647080.0,2647080.0,2.390185,2.390185,2.390185,2.390185,NaN,0


In [85]:
#train_data = train_data.drop(['alert_key'], axis =1).drop_duplicates()

In [86]:
Y = train_data['y']
X = train_data[x_col]

In [87]:
X

,trans_no,trade_amount_usd,total_amt_rolling_3,on_day_total_amt,total_amt_rolling_7,total_amt_rolling_14,ratio_total_aum_7,ratio_total_aum_1,ratio_total_aum_14,ratio_total_aum_3,over2
0,4,10309410.0,10309410.0,10309410.0,10309410.0,10309410.0,4.847146,4.847146,4.847146,4.847146,0.0
1,4,10309410.0,10309410.0,10309410.0,10309410.0,10309410.0,10.513402,10.513402,10.513402,10.513402,0.0
2,4,10309410.0,10309410.0,10309410.0,10309410.0,10309410.0,10.513402,10.513402,10.513402,10.513402,0.0
3,4,10309410.0,10309410.0,10309410.0,10309410.0,10309410.0,10.513402,10.513402,10.513402,10.513402,0.0
4,0,63270.0,63270.0,63270.0,63270.0,63270.0,0.034812,0.034812,0.034812,0.034812,NaN
...,...,...,...,...,...,...,...,...,...,...,...
18396,0,996270.0,996270.0,996270.0,996270.0,996270.0,434.483210,434.483210,434.483210,434.483210,NaN
18397,0,996270.0,996270.0,996270.0,996270.0,996270.0,434.483210,434.483210,434.483210,434.483210,NaN
18398,0,2647080.0,2647080.0,2647080.0,2647080.0,2647080.0,24.776114,24.776114,24.776114,24.776114,NaN
18399,0,2647080.0,2647080.0,2647080.0,2647080.0,2647080.0,2.390185,2.390185,2.390185,2.390185,NaN


In [88]:
X.replace([np.inf, -np.inf], 1, inplace=True)

C:\Users\rocker\anaconda3\lib\site-packages\pandas\core\frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [89]:
#model training 1
#逐筆交易處理
test_size = 0.2
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0,
        enable_categorical=True,
        use_label_encoder=False,
        predictor= "gpu_predictor")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
#previous 30days label of 1
# evaluate predictions
f1 = f1_score(y_test, predictions)
print("F1 score: %.2f%%" % (f1 * 100.0))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
#F1 score: 73.76%
#0.9921875
#0.5869618097786427

F1 score: 95.33%
0.9683972911963883
0.9387308533916849


In [90]:
pd.DataFrame({'columns':list(X.columns),'score':model.feature_importances_})

,columns,score
0,trans_no,0.026006
1,trade_amount_usd,0.035017
2,total_amt_rolling_3,0.023310
3,on_day_total_amt,0.161323
4,total_amt_rolling_7,0.026067
5,total_amt_rolling_14,0.024911
6,ratio_total_aum_7,0.020680
7,ratio_total_aum_1,0.059187
8,ratio_total_aum_14,0.019188
9,ratio_total_aum_3,0.020213


In [91]:
#彙整model training 1偵測結果
#predict_proba以0.05機率區間為一個欄位判斷分佈
def mapping(proba):
    if proba <= 0.05:
        return 1
    elif 0.05 < proba <= 0.1:
        return 2
    elif 0.1 < proba <= 0.15:
        return 3
    elif 0.15 < proba <= 0.2:
        return 4
    elif 0.2 < proba <= 0.25:
        return 5
    elif 0.25 < proba <= 0.3:
        return 6
    elif 0.3 < proba <= 0.35:
        return 7
    elif 0.35 < proba <= 0.4:
        return 8
    elif 0.4 < proba <= 0.45:
        return 9
    elif 0.45 < proba <= 0.5:
        return 10
    elif 0.5 < proba <= 0.55:
        return 11
    elif 0.55 < proba <= 0.6:
        return 12
    elif 0.6 < proba <= 0.65:
        return 13
    elif 0.65 < proba <= 0.7:
        return 14
    elif 0.7 < proba <= 0.75:
        return 15
    elif 0.75 < proba <= 0.8:
        return 16
    elif 0.8 < proba <= 0.85:
        return 17
    elif 0.85 < proba <= 0.9:
        return 18
    elif 0.9 < proba <= 0.95:
        return 19
    elif 0.95 < proba <= 1:
        return 20

In [92]:
train_data['proba'] = model.predict_proba(X)[:,1]

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:
result = train_data[['cust_id','proba','y']]
result['level'] = result["proba"].map(mapping)
result = result[['cust_id', 'level', 'y']]
result = result.groupby(['cust_id','level']).count().reset_index().pivot_table(index='cust_id', columns='level', values='y')
result.fillna(0, inplace=True)
result = result.div(result.sum(axis=1), axis=0)
result = result.reset_index()
for col in list(set([num for num in range(1,21,1)]) -  set(result.columns[1:])):
  result[col] = 0.0
result = result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
result = result.merge(train_data[['cust_id','y']].drop_duplicates(), on='cust_id', how='left')
result.pop('cust_id')

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0      002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...
1      0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...
2      0068c614b131d745d71672044f1cc660ca30dd71e48338...
3      00b342a67f967d662c851893d49d3da0716041fac5fa90...
4      00eabc23fbb38516c8e9a13f3a8f8d00579ff7baeb2ec8...
                             ...                        
949    fee9c88690656f65f8829b3d3e6082b3b7b58ecbb56adb...
950    ff38c93051cf7c0175c67f5171271d4b8c8bb40533df65...
951    ff4fa0c71df2a8a9074c1b8ad537e0f15c93ad16e1165e...
952    ff6b0040005b54f4b7955942e74a0a9ec466abb2a017a5...
953    ff6c1217b5e039fda46bf4dfba2d54291154a1db54c677...
Name: cust_id, Length: 954, dtype: object

In [94]:
#model training 2
#歸戶判斷是否報SAR
result_col = list(result.columns)
result_col.remove('y')
test_size = 0.2
seed = 42
X_train, X_test, y_train, y_test = train_test_split(result[result_col], result['y'], test_size=test_size, random_state=seed)
model_2nd = XGBClassifier()
model_2nd.fit(X_train, y_train)
y_pred = model_2nd.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
f1 = f1_score(y_test, predictions)
print("F1 score: %.2f%%" % (f1 * 100.0))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
#F1 score: 36.36%
#0.7142857142857143
#0.24390243902439024

F1 score: 83.33%
0.9375
0.75


In [95]:
alert_remit

,cust_id,trans_date,trans_no,trade_amount_usd,alert_key,tx_date,risk_rank,occupation_code,total_asset,AGE,...,alert_key_1_1,total_amt_rolling_7,cust_id_1,alert_key_1,total_amt_rolling_14,ratio_total_aum_7,ratio_total_aum_1,ratio_total_aum_14,ratio_total_aum_3,over2
0,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,389,0,904740.0,360990,389,3,12.0,703297.0,5,...,360990.0,904740.0,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,360990.0,904740.0,1.286427,1.286427,1.286427,1.286427,0.0
1,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,390,4,570000.0,360990,389,3,12.0,703297.0,5,...,356602.0,570000.0,0388c9d9ef39aad168e450e671382cfa405d61efd39517...,356602.0,570000.0,0.810468,0.810468,0.810468,0.810468,NaN
2,0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...,176,3,250680.0,352282,365,1,12.0,7450.0,5,...,356602.0,250680.0,0388c9d9ef39aad168e450e671382cfa405d61efd39517...,356602.0,250680.0,33.648322,33.648322,33.648322,33.648322,NaN
3,0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...,176,3,250680.0,357262,370,1,12.0,7450.0,5,...,356602.0,250680.0,0388c9d9ef39aad168e450e671382cfa405d61efd39517...,356602.0,250680.0,33.648322,33.648322,33.648322,326.818792,NaN
4,0388c9d9ef39aad168e450e671382cfa405d61efd39517...,363,0,604530.0,356602,378,1,19.0,14128.0,3,...,356602.0,604530.0,0388c9d9ef39aad168e450e671382cfa405d61efd39517...,356602.0,604530.0,42.789496,42.789496,42.789496,175.415487,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6721,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,393,0,259260.0,357341,370,2,17.0,41066.0,3,...,NaN,259260.0,NaN,NaN,259260.0,6.313252,6.313252,6.313252,6.313252,1.0
6722,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,393,0,259260.0,358624,384,2,17.0,41066.0,3,...,NaN,259260.0,NaN,NaN,259260.0,6.313252,6.313252,6.313252,6.313252,1.0
6723,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,393,0,259260.0,361459,390,2,17.0,41066.0,3,...,NaN,259260.0,NaN,NaN,259260.0,6.313252,6.313252,6.313252,6.313252,1.0
6724,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,393,0,259260.0,363341,379,2,17.0,41066.0,3,...,NaN,259260.0,NaN,NaN,259260.0,6.313252,6.313252,6.313252,6.313252,1.0


In [52]:
alert_result.to_csv('remit_predict.csv', index=False)

In [96]:
#Submission
alert_x = alert_remit[x_col]
alert_data = alert_remit
alert_data['proba'] = model.predict_proba(alert_x)[:,1]
alert_result = alert_data[['cust_id','proba']]
alert_result['level'] = alert_result["proba"].map(mapping)
alert_result = alert_result[['cust_id', 'level']]
alert_result['cnt'] = 1
alert_result = alert_result.groupby(['cust_id','level'])['cnt'].count().reset_index().pivot_table(index='cust_id', columns='level', values='cnt')
alert_result.fillna(0, inplace=True)
alert_result = alert_result.div(alert_result.sum(axis=1), axis=0)
alert_result = alert_result.reset_index()
for col in list(set([num for num in range(1,21,1)]) -  set(alert_result.columns[1:])):
  alert_result[col] = 0.0
alert_result = alert_result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
#alert_result.pop('cust_id')
alert_pred = model_2nd.predict_proba(alert_result[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]])
# evaluate predictions
final = alert_result[['cust_id']]
final['probability'] = alert_pred[:,1]
final

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


level,cust_id,probability
0,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,0.001620
1,0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...,0.044000
2,0388c9d9ef39aad168e450e671382cfa405d61efd39517...,0.205782
3,03d4cbfcb065501bebb2b75ea4df2b779df0192a183af3...,0.001620
4,0658224f10ea4c5c8b4252f1bef32ee1071dc504bfc670...,0.001620
...,...,...
185,f8ce663f765e2459578d285a032a16046fba0b2bba8e12...,0.978981
186,fa1919bc7f2d48288a1eab1b97b2a495557a27789cbea7...,0.515277
187,fe0b3b09a3b8c5435d55a286ecd4fb416e96d3231abdf2...,0.001620
188,fe69b536bfc0589e8198533243f08b2b9401bc352039f9...,0.057373


In [97]:
final = alert_time.merge(cdtx2[['alert_key', 'cust_id']].merge(final, on='cust_id'), on='alert_key', how='left')[['alert_key', 'probability']]

In [98]:
final.fillna(0, inplace=True)
final

,alert_key,probability
0,352249,0.000000
1,352253,0.001620
2,352254,0.000000
3,352280,0.001294
4,352282,0.044000
...,...,...
1840,365001,0.000000
1841,365004,0.001620
1842,365008,0.001540
1843,365009,0.000000


In [99]:
doc = doc[['alert_key']]
doc

,alert_key
0,357307
1,376329
2,373644
3,357668
4,354443
...,...
3845,364485
3846,363155
3847,368710
3848,358067


In [100]:
final = doc.merge(final, on='alert_key', how='left')
final.fillna(0,inplace=True)
final

,alert_key,probability
0,357307,0.000000
1,376329,0.000000
2,373644,0.000000
3,357668,0.013029
4,354443,0.001558
...,...,...
3845,364485,0.000000
3846,363155,0.002165
3847,368710,0.000000
3848,358067,0.001620


In [101]:
final.loc[final[final['alert_key'].isin(prev_list)].index,'probability']=0

In [103]:
final.to_csv('remit_aml.csv')

In [102]:
final[final['probability']>0]

,alert_key,probability
3,357668,0.013029
5,361128,0.002916
6,364753,0.979861
9,360609,0.001620
143,361314,0.001620
...,...,...
3729,364718,0.002348
3737,361269,0.001620
3741,354606,0.001620
3755,364472,0.001620
